# Create a SQL-lite database to hold our csv files

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, Float

from sqlalchemy.orm import Session

# Our Create Statement
engine = create_engine("sqlite:///hawaii.sqlite")


## Creating classes for the stations and measures tables

In [2]:
class stations(Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float(64))
    longitude = Column(Float(64))
    elevation = Column(Float(64))
    
class measures(Base):
    __tablename__ =  'measures'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float(64))
    tobs = Column(Integer)
    
    
    

In [3]:
#sending the tables to our SQL Lite Server
Base.metadata.create_all(engine)

# Making it ready for manipulation
session = Session(bind=engine)

### Load the data using Pandas

In [4]:
import pandas as pd
stations_df = pd.read_csv('resources/hawaii_stations.csv', encoding='utf-8',)
measures_df = pd.read_csv('resources/clean_measures.csv',encoding='utf-8', index_col='Unnamed: 0').drop('nulls', axis=1)

In [8]:
measures_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19550 entries, 0 to 19549
Data columns (total 4 columns):
station    19550 non-null object
date       19550 non-null object
prcp       19550 non-null float64
tobs       19550 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 763.7+ KB


## Send our CSV data to the sql server

In [6]:
for row in stations_df.iterrows():
    session.add(stations(id=int(row[0]),station=str(row[1]['station']), name=str(row[1]['name']), latitude=row[1]['latitude'], 
                         longitude=row[1]['longitude'], elevation=row[1]['elevation'],))
session.commit()    

In [9]:
for row in measures_df.iterrows():
    session.add(measures(id=int(row[0]),station=row[1]['station'], date=row[1]['date'], prcp=row[1]['prcp'],
                        tobs=row[1]['tobs']))
session.commit()

## Check and see if we can access it 

In [10]:
pd.read_sql("SELECT * FROM measures", engine).head()

,id,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,4,USC00519397,2010-01-06,0.03,73
